### In order to debug the torch transfer of weights, we're gonna side by side compare the forward passes of the two models... :(

Start with loading keras generator network

In [1]:
import sys
# point path to genesis repo
sys.path.insert(
    0,
    '/gpfs/commons/home/tchen/al_project/genesis/analysis/splicing'
)
from definitions.generator.splirent_deconv_conv_generator_concat import load_generator_network
from genesis.generator import build_generator, st_sampled_softmax, st_hardmax_softmax
from pathlib import Path
from keras.models import load_model
from keras import backend as K
import numpy as np
from keras.layers import Dense, Dropout, Activation, Flatten, Input, Lambda, Concatenate, Reshape, Softmax, Conv2DTranspose
from keras import Model

Using TensorFlow backend.


In [2]:
model_name = 'genesis_splicing_cnn_target_isoform_00_pwm_and_multisample_hek_only_random_regions_50_epochs_harderentropy_generator.h5'
model_save_dir = '/gpfs/commons/groups/knowles_lab/ting/DEN_splicing_pretrained_models/'

full_path = model_save_dir + model_name
generator_model = load_model(filepath=str(full_path), custom_objects={'K': K, 'st_sampled_softmax': st_sampled_softmax, 'st_hardmax_softmax': st_hardmax_softmax}, compile=False)

2024-03-11 11:19:46.052089: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 AVX512F FMA
2024-03-11 11:19:46.062517: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2400000000 Hz
2024-03-11 11:19:46.064659: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x5599d4b78d70 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2024-03-11 11:19:46.064693: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version


Instructions for updating:
Use `tf.random.categorical` instead.



Now we load the torch model with the previously saved weights

In [52]:
import sys
# point path to our repo
sys.path.insert(
    0,
    '/gpfs/commons/home/tchen/al_project/active-learning-cnns-gps/src'
)

import numpy as np
from models.den import Generator
import torch
from torch import nn
import torch.nn.functional as F

In [4]:
save_path = '/gpfs/commons/groups/knowles_lab/ting/DEN_splicing_generator_weights/'
save_name = 'target_isoform_00.npy'

target_isoform_net_weights = np.load(save_path+save_name, allow_pickle=True)

# list of numpy arrays
weights = target_isoform_net_weights.tolist()
device = torch.device('cpu')

# need to extract embedding template and mask from pretrained network
pretrained_embedding_template = torch.tensor(weights[-2]).to(device)
pretrained_embedding_mask = torch.tensor(weights[-1]).to(device)

n_samples = 10
n_classes = 1
seq_length = 109
latent_dim = 100
batch_size = 32

torch_generator = Generator(embedding_template=pretrained_embedding_template,
                                   embedding_mask=pretrained_embedding_mask,
                                   device=device,
                                   latent_dim=latent_dim,
                                   batch_size=batch_size,
                                   seq_length=seq_length,
                                   n_classes=n_classes,
                                   n_samples=n_samples)


In [5]:

running_keras_weight_idx = 0

for i, layer in enumerate(torch_generator.generator_network.generator_network):
    if isinstance(layer, nn.Linear):
        # transfer linear layer weights and biases
        assert tuple(layer.weight.data.shape) == weights[running_keras_weight_idx].T.shape
        layer.weight.data = torch.from_numpy(weights[running_keras_weight_idx].T)
        running_keras_weight_idx += 1

        assert layer.bias.data.shape == weights[running_keras_weight_idx].shape
        layer.bias.data = torch.from_numpy(weights[running_keras_weight_idx])
        running_keras_weight_idx += 1
    elif isinstance(layer, nn.ConvTranspose2d):
        # transfer convtranspose2d weights and biases
        assert tuple(layer.weight.data.shape) == np.transpose(weights[running_keras_weight_idx], axes=[3, 2, 0, 1]).shape
        layer.weight.data = torch.from_numpy(np.transpose(weights[running_keras_weight_idx], axes=[3, 2, 0, 1]))
        running_keras_weight_idx += 1
        
        assert layer.bias.data.shape == weights[running_keras_weight_idx].shape
        layer.bias.data = torch.from_numpy(weights[running_keras_weight_idx])
        running_keras_weight_idx += 1
    elif isinstance(layer, nn.Conv2d):
        # transfer conv2d weights and biases
        assert tuple(layer.weight.data.shape) == np.transpose(weights[running_keras_weight_idx], axes=[3, 2, 0, 1]).shape
        layer.weight.data = torch.from_numpy(np.transpose(weights[running_keras_weight_idx], axes=[3, 2, 0, 1]))
        running_keras_weight_idx += 1
        
        assert layer.bias.data.shape == weights[running_keras_weight_idx].shape
        layer.bias.data = torch.from_numpy(weights[running_keras_weight_idx])
        running_keras_weight_idx += 1
    elif isinstance(layer, nn.BatchNorm2d):
        # transfer batch norm gamma, beta, running mean, running variance
        # order from keras model should be gamma, beta, moving mean, moving variance
        assert tuple(layer.weight.data.shape) == weights[running_keras_weight_idx].shape # this might need a transpose
        layer.weight.data = torch.from_numpy(weights[running_keras_weight_idx])
        running_keras_weight_idx += 1
        
        assert layer.bias.data.shape == weights[running_keras_weight_idx].shape
        layer.bias.data = torch.from_numpy(weights[running_keras_weight_idx])
        running_keras_weight_idx += 1
        
        assert layer.running_mean.data.shape == weights[running_keras_weight_idx].shape
        layer.running_mean.data = torch.from_numpy(weights[running_keras_weight_idx])
        running_keras_weight_idx += 1
        
        assert layer.running_var.data.shape == weights[running_keras_weight_idx].shape
        layer.running_var.data = torch.from_numpy(weights[running_keras_weight_idx])
        running_keras_weight_idx += 1

In [6]:
torch_generator.to(device)
torch_generator.eval()

Generator(
  (generator_network): GeneratorNetwork(
    (generator_network): ModuleList(
      (0): Linear(in_features=101, out_features=3456, bias=True)
      (1): ConvTranspose2d(384, 256, kernel_size=(7, 1), stride=(2, 1))
      (2): BatchNorm2d(256, eps=1e-05, momentum=0.99, affine=True, track_running_stats=True)
      (3): ConvTranspose2d(256, 192, kernel_size=(8, 1), stride=(2, 1))
      (4): BatchNorm2d(192, eps=1e-05, momentum=0.99, affine=True, track_running_stats=True)
      (5): ConvTranspose2d(192, 128, kernel_size=(7, 1), stride=(2, 1))
      (6): BatchNorm2d(128, eps=1e-05, momentum=0.99, affine=True, track_running_stats=True)
      (7): Conv2d(128, 128, kernel_size=(8, 1), stride=(1, 1), padding=same)
      (8): BatchNorm2d(128, eps=1e-05, momentum=0.99, affine=True, track_running_stats=True)
      (9): Conv2d(128, 64, kernel_size=(8, 1), stride=(1, 1), padding=same)
      (10): BatchNorm2d(64, eps=1e-05, momentum=0.99, affine=True, track_running_stats=True)
      (11): 

Okay now we'll start with the same input and layer by layer compare the outputs of the keras model with the outputs of the torch model

In [22]:
# generator forward loop (up until generator network)

# Seed class input for all dense/embedding layers
sequence_class = torch.empty(batch_size).to(device)

# TODO: Super jank, refactor this...
sequence_class.uniform_(-0.499, 1-0.5001)
sequence_class = torch.round(sequence_class).int()
# sequence_class = torch.ones(self.batch_size, dtype=torch.int32).to(self.device)

# Get generated policy pwm logits (non-masked)
# latent_input_1 = torch.empty((batch_size, latent_dim)).to(device)
# latent_input_2 = torch.empty((batch_size, latent_dim)).to(device)
latent_input_1 = torch.ones((batch_size, latent_dim)).to(device)
latent_input_2 = torch.ones((batch_size, latent_dim)).to(device)

# latent_input_1.uniform_(-1.0, 1.0)
# latent_input_2.uniform_(-1.0, 1.0)

sequence_class_onehots = torch.eye(1).to(device)

# check here for bugs
class_embedding = sequence_class_onehots.index_select(0, index=sequence_class)    # tf.gather equivalent
# class_embedding = torch.ones((self.batch_size, 1), dtype=torch.int32).to(self.device)

torch_seed_input_1 = torch.cat([latent_input_1, class_embedding], dim=-1)
torch_seed_input_2 = torch.cat([latent_input_2, class_embedding], dim=-1)

In [8]:
# repeat same for keras model
sequence_class_input = Input(tensor=K.ones((batch_size, 1)), dtype='int32', name='sequence_class_seed')
sequence_class = Lambda(lambda inp: K.cast(K.round(inp * K.random_uniform((batch_size, 1), minval=-0.4999, maxval=n_classes-0.5001)), dtype='int32'), name='lambda_rand_sequence_class')(sequence_class_input)

sequence_class_onehots = np.eye(n_classes)

#Generator network parameters
latent_size = 100

#Generator inputs
latent_input_1 = Input(tensor=K.ones((batch_size, latent_size)), name='noise_input_1')
latent_input_2 = Input(tensor=K.ones((batch_size, latent_size)), name='noise_input_2')
# latent_input_1_out = Lambda(lambda inp: inp * K.random_uniform((batch_size, latent_size), minval=-1.0, maxval=1.0), name='lambda_rand_input_1')(latent_input_1)
# latent_input_2_out = Lambda(lambda inp: inp * K.random_uniform((batch_size, latent_size), minval=-1.0, maxval=1.0), name='lambda_rand_input_2')(latent_input_2)

class_embedding = Lambda(lambda x: K.gather(K.constant(sequence_class_onehots), K.cast(x[:, 0], dtype='int32')))(sequence_class)

keras_seed_input_1 = Concatenate(axis=-1)([latent_input_1, class_embedding])
keras_seed_input_2 = Concatenate(axis=-1)([latent_input_2, class_embedding])

In [10]:
func = K.function([sequence_class_input, latent_input_1, latent_input_2], [keras_seed_input_1, keras_seed_input_2])

In [11]:
keras_sequence_class = func([K.ones((batch_size, 1)), K.ones((batch_size, latent_size)), K.ones((batch_size, latent_size))])

All good so far from here, let's start testing the generator networks layer by layer

In [54]:
torch_dense_output = F.relu(torch_generator.generator_network.generator_network[0](torch_seed_input_1))

In [35]:
# copy dense layer weights over
layer_input = Input(shape=(latent_dim+1,))
copy_dense_1 = Dense(9 * 384, activation='relu', kernel_initializer='glorot_uniform', name='policy_dense_1')
out = copy_dense_1(layer_input)
model = Model(inputs=layer_input, outputs=out)
copy_dense_1.set_weights(generator_model.get_layer('policy_dense_1').get_weights())

In [55]:
keras_dense_output = model.predict(keras_sequence_class[0])

In [74]:
np.testing.assert_allclose(torch_dense_output.detach().numpy(), keras_dense_output) 

First dense layer passed

In [72]:
torch_deconv_0_output = torch_generator.generator_network.generator_network[1](torch_dense_output.reshape(batch_size, 384, 9, 1))

In [93]:
layer_input = Input(shape=(9*384,))
policy_dense_1_reshape = Reshape((9, 1, 384))
policy_deconv_0 = Conv2DTranspose(256, (7, 1), strides=(2, 1), padding='valid', activation='linear', kernel_initializer='glorot_normal', name='policy_deconv_0')
out = policy_deconv_0(policy_dense_1_reshape(layer_input))
model = Model(inputs=layer_input, outputs=out)
policy_deconv_0.set_weights(generator_model.get_layer('policy_deconv_0').get_weights())

In [94]:
keras_deconv_0_output = model.predict(keras_dense_output)

In [102]:
keras_deconv_0_output.shape

(32, 23, 1, 256)

In [76]:
torch_deconv_0_output.detach().numpy().shape == keras_deconv_0_output.transpose([0, 3, 1, 2]).shape

True

In [96]:
np.testing.assert_allclose(torch_deconv_0_output.detach().numpy(), keras_deconv_0_output.transpose([0, 3, 1, 2])) 

AssertionError: 
Not equal to tolerance rtol=1e-07, atol=0

Mismatched elements: 188416 / 188416 (100%)
Max absolute difference: 57.36231
Max relative difference: 850.8875
 x: array([[[[  6.582627],
         [ -1.922581],
         [  1.532796],...
 y: array([[[[ 2.139414e+00],
         [-3.674655e-03],
         [ 5.357711e+00],...

Looks like the error was in the conv2dTranspose...

In [98]:
deconv_0 = nn.ConvTranspose2d(in_channels=384, out_channels=256, kernel_size=(7, 1), stride=(2, 1))

In [101]:
generator_model.get_layer('policy_deconv_0').get_weights()[0].shape # kernel height, width, out channels, in channels

(7, 1, 256, 384)

In [100]:
deconv_0.weight.shape # in channels, out channels, kernel height, kernel width

torch.Size([384, 256, 7, 1])

In [90]:
# check weights are equal
np.testing.assert_allclose(torch_generator.generator_network.generator_network[1].bias.detach().numpy(), generator_model.get_layer('policy_deconv_0').get_weights()[1]) 

In [92]:
np.testing.assert_allclose(torch_generator.generator_network.generator_network[1].weight.detach().numpy(), generator_model.get_layer('policy_deconv_0').get_weights()[0].transpose([3, 2, 0, 1])) 